In [1]:
from utilities import * 
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
%load_ext autoreload
%autoreload 2
nest_asyncio.apply()

df1_path = '../dataset/result/nist_form/20230906073654_14/test_pred.csv'
df2_path = '../dataset/raw/nist/aws_cleaned.csv'

output_path = '../dataset/raw/nist/donut-tt-gpt_ver1.csv'

## Donut result:

In [2]:
df1 = pd.read_csv(df1_path)
df1.fillna('None', inplace=True)
print(df1.shape)
df1.head()

(500, 5)


,DATE,CITY,STATE,ZIP,image_name
0,08/02/89,FLINT,Mi,48504,f0000_14.png
1,8-7-89,Allendale,MI,49401,f0001_41.png
2,08-03-89,Holland,Mi,49424,f0004_09.png
3,8/3/89,ST. CLAIR SHORES,MI.,48082,f0006_12.png
4,8/2/89,EAST LANSING,MicHIGAN,48823,f0009_06.png


## TT result:


In [3]:
df2 = pd.read_csv(df2_path)
df2.fillna('None', inplace=True)
print(df2.shape)
df2.head()

(2100, 9)


,image_name,DATE,DATE__CONFIDENCE,CITY,CITY__CONFIDENCE,STATE,STATE_CONFIDENCE,ZIP,ZIP_CONFIDENCE
0,f0000_14.png,08/02/89,88.97464,FLiNT,86.739792,Mi,80.102089,48504,73.823257
1,f0001_41.png,8-7-89,87.28508,Allendale,89.946899,MI,82.12265,49401,77.600266
2,f0002_01.png,8-3-89,81.038841,MINDEN CITY,80.873611,mi,35.721474,48456,37.457157
3,f0003_42.png,08-03-89,87.377403,Charlotte,80.985222,MI,77.612007,48813,71.598038
4,f0004_09.png,08-03-89,89.594048,Holland,80.345238,Mi,62.762756,49424,58.702671


# Donut + TT + GPU

In [4]:
def clean_jsons(json1, json2):
    
    prompt = f"""Use the following examples of JSONs to 
    convert the two input JSONs delimited by triple backticks into one output JSON.

    example1: 
    
    JSON1 = {{'DATE': '8-10-89', 'CITY': None, 'STATE': 'Sacramento Ca', 'ZIP': '95841'}}
    JSON2 = {{'DATE': '8-10-89', 'CITY': 'Sacrament', 'STATE': 'None', 'ZIP': 'Ca95841'}}

    OUTPUT_JSON = {{'DATE': 'Sacramento', 'CITY': 'Sacramento', 'STATE': 'Ca', 'ZIP': '95841'}}
    
    example2: 

    JSON1 = {{'DATE': '9-21-89', 'CITY': 'Sonora, Texas', 'STATE': '76950', 'ZIP': None}}
    JSON2 = {{'DATE': '9-21-89', 'CITY': 'Sonora,', 'STATE': 'Texas 76950', 'ZIP': None}}

    OUTPUT_JSON = {{'DATE': '9-21-89', 'CITY': 'Sonora', 'STATE': 'Texas', 'ZIP': '76950'}}
    
    
    example3: 

    JSON1 = {{'DATE': '9/27/89', 'CITY': 'Austin', 'STATE': 'MN', 'ZIP': '55912'}}
    JSON2 = {{'DATE': '9/27/89', 'CITY': 'Austin', 'STATE': 'MN', 'ZIP': '55412'}}

    OUTPUT_JSON = {{'DATE': '9/27/89', 'CITY': 'Austin', 'STATE': 'MN', 'ZIP': '55912'}}
    
    example4: 
    
    JSON1 = {{'DATE': '09-17-89', 'CITY': 'Brownsville', 'STATE': 'TX. 78521', 'ZIP': None}}
    JSON2 = {{'DATE': '09-17-89', 'CITY': 'BROWnsville', 'STATE': None, 'ZIP': None}}

    OUTPUT_JSON = {{'DATE': '09-17-89', 'CITY': 'BROWnsville', 'STATE': 'TX.', 'ZIP': '78521'}}
    
    
    example5: 
    
    JSON1 = {{'DATE': 'Aug. 4,1989', 'CITY': 'Marietta, Ohio', 'STATE': '45754', 'ZIP': None}}
    JSON2 = {{'DATE': 'Aug. 4,1989', 'CITY': 'Marietta, Ohio', 'STATE': 'None', 'ZIP': '45754'}}

    OUTPUT_JSON = {{'DATE': 'Aug. 4,1989', 'CITY': 'Marietta', 'STATE': 'Ohio', 'ZIP': '45754'}}
    

    Make sure that the CITY is in the correct STATE.
    Make sure that the STATE is in the correct CITY. 
    STATE value cannot and should not contain any numbers or digits.
    Don't change the STATE value from abbreviation to complete format.
    ZIP is zip code and contains only numbers. If you got a ZIP with a punctuation remove the punctuation.
    ZIP is zip code and does not contain /, and / must be changed to 1.
    
    In the STATE field, if you got two words and the second one was an abbreviation of a 
    state name in the United States of America, the first word should be part of the  CITY field.
    Similar to example5
    
    example5: 
    
    JSON1 = {{'DATE': '9/11/89', 'CITY': 'Richland', 'STATE': 'Center Wi 53581', 'ZIP': None}}
    JSON2 = {{'DATE': '9/4189', 'CITY': 'Rich land', 'STATE': 'Center Wi', 'ZIP': '53581'}}

    OUTPUT_JSON = {{'DATE': '9/21/89', 'CITY': 'Richland Center', 'STATE': 'Wi', 'ZIP': '53581'}}
    
    YOU MUST FOLLOW THE NEXT 8 RULS:
    
    0. DO NOT CHANGE LETTER CASING OF THE INPUT.
    
    1. If the STATE field had a sequence of numbers, they must be moved to the ZIP field. 
    
    2. If the STATE field appears as the full version of the state name and not the abbreviation, keep the full version.     
    
    3. If the STATE field letters apears as capitalised, DO NOT convert the letters to small letters. 
    
    4. If the CITY field letters are capitalised, DO NOT convert the letters to small letters and They MUST kepet as capitalised.
    
    5. DO NOT convert the letters from upper case to lower case. DO NOT change lower case to upper case. 
    
    6. Check the validity of values in the STATE and CITY fields. For example, if the value for the CITY was "ROMEOUILLE" and the value for the STATE field was "ILL",  then the correct spelling for the  CITY field should be "ROMEOVILLE". In such cases, correct the spelling of the CITY field. 

    7. Check the validity of values in the STATE and CITY fields. For example, if the value for the CITY was "New York City" and the STATE field was "NU" filed the spelling for the STATE field and update it to "NY".
    
    OUTPUT_JSON:
    Format your response as a JSON object with all four `keys'.
    OUTPUT_JSON reply MUST be a single json with the following KEYS:

    - DATE
    - CITY
    - STATE
    - ZIP
    
    REPLY ONLY A SINGLE JSON WITH: DATE, CITY ,STATE, ZIP OR I WILL KILL YOU
    REPLY ONLY A SINGLE JSON WITH: DATE, CITY ,STATE, ZIP OR I WILL KILL YOU
    IF YOU CANT MAKE IT TO ONE JSON JUST RETURN THE json2
    Return a single json and don not assign a name to it like `OUTPUT_JSON =`. 
    Return a single json that it can be loaded using json.loads(YOUR_RESULT) in python.
    
    text: ```JSON1 = {json1}, JSON1 = {json2}```
    """

    return get_completion(prompt)

In [5]:
def clean_dicts_(input_image_name):
    
    # collect the JSONS
    df_1 = df1[df1.image_name == input_image_name]
    df_2 = df2[df2.image_name == input_image_name]
    
    # make sure they both JSONS have all the keys
    dict1 = init_json(df_1)
    dict2 = init_json(df_2)
    
    # creat the promet
    dict_ = clean_jsons(dict1, dict2)
    dict3 = json.loads(dict_)
    
    return dict1, dict2, dict3

async def run_function_with_retry(def_to_run, 
                                  input_image_name,
                                  max_retry_count= 2,
                                  retry_interval= 1):

    for retry in range(max_retry_count):
        loop = asyncio.get_event_loop()
        with concurrent.futures.ThreadPoolExecutor() as pool:
            try:
                future = pool.submit(def_to_run, input_image_name)
                result = await asyncio.wait_for(loop.run_in_executor(None, future.result), timeout=10)
                return result 
                break
            except asyncio.TimeoutError:
                print("Function didn't finish in time. Retrying...")
            except concurrent.futures.TimeoutError:
                print("Concurrent Future didn't finish in time. Retrying...")
            await asyncio.sleep(retry_interval)
    else:
        print("Function didn't finish within retry attempts.")

In [6]:
error_list = []
result_list = []
for i in tqdm(df1.image_name.to_list()):
    try:
        loop = asyncio.get_event_loop()
        dict1, dict2, dict3  = loop.run_until_complete(run_function_with_retry(clean_dicts_, i))
        dict3['image_name'] = i
        result_list.append(dict3)

    except Exception as e:
        print(str(e))
        error_list.append(i)
    

 91%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏              | 453/500 [18:13<01:34,  2.02s/it]

Function didn't finish in time. Retrying...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [30:06<00:00,  3.61s/it]


In [7]:
len(error_list)

0

In [8]:
df_gpt = pd.DataFrame(result_list)
print(df_gpt.shape)
df_gpt.head(2)

(500, 5)


,DATE,CITY,STATE,ZIP,image_name
0,08/02/89,FLINT,Mi,48504,f0000_14.png
1,8-7-89,Allendale,MI,49401,f0001_41.png


In [9]:
error_list_2 = []
for i in tqdm(error_list):
    try:
        loop = asyncio.get_event_loop()
        dict1, dict2, dict3  = loop.run_until_complete(run_function_with_retry(clean_dicts_, i))
        dict3['image_name'] = i
        result_list.append(dict3)

    except Exception as e:
        print(str(e))
        error_list_2.append(i)

0it [00:00, ?it/s]


In [10]:
df_gpt = pd.DataFrame(result_list)
df_gpt.drop_duplicates(inplace = True)
print(df_gpt.shape)
df_gpt.head(2)

(500, 5)


,DATE,CITY,STATE,ZIP,image_name
0,08/02/89,FLINT,Mi,48504,f0000_14.png
1,8-7-89,Allendale,MI,49401,f0001_41.png


In [11]:
df_gpt.to_csv(output_path, index = False)